In [1469]:
import csv
import collections
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint
global FEATURE_TYPES


In [1470]:
df = pd.read_csv('Job_Placement_Data.csv')
df = df.drop(["ssc_board","hsc_board","hsc_subject","gender"], axis=1)
# df["status"] = pd.Categorical(df["status"]).codes
# df["work_experience"] = pd.Categorical(df["work_experience"]).codes
# df["gender"] = pd.Categorical(df["gender"]).codes
# df["specialisation"] = pd.Categorical(df["specialisation"]).codes
# df = df.drop("ssc_board", axis=1)
# df = df.drop("hsc_board", axis=1)
# df = df.drop("ssc_percentage",axis=1)
# df = df.drop(["hsc_percentage","hsc_subject","degree_percentage", "undergrad_degree"], axis=1)

In [1471]:
df.head()

,ssc_percentage,hsc_percentage,degree_percentage,undergrad_degree,work_experience,emp_test_percentage,specialisation,mba_percent,status
0,67.00,91.00,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,79.33,78.33,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,65.00,68.00,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,56.00,52.00,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,85.80,73.60,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed


In [1472]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ssc_percentage       215 non-null    float64
 1   hsc_percentage       215 non-null    float64
 2   degree_percentage    215 non-null    float64
 3   undergrad_degree     215 non-null    object 
 4   work_experience      215 non-null    object 
 5   emp_test_percentage  215 non-null    float64
 6   specialisation       215 non-null    object 
 7   mba_percent          215 non-null    float64
 8   status               215 non-null    object 
dtypes: float64(5), object(4)
memory usage: 15.2+ KB


# Train Test Split

In [1473]:
def train_test_split(df, test_size):
    if isinstance(test_size,float):
        test_size = round(test_size*len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population = indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    # val_df = train_df

    return train_df, test_df

In [1474]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size = 0.1)

In [1475]:
# sns.lmplot(data=train_df, x="work_experience", y="mba_percent", hue="status")

# Helper Functions

In [1476]:
data  = train_df.values
data[:5]

array([[67.0, 91.0, 58.0, 'Sci&Tech', 'No', 55.0, 'Mkt&HR', 58.8,
        'Placed'],
       [79.33, 78.33, 77.48, 'Sci&Tech', 'Yes', 86.5, 'Mkt&Fin', 66.28,
        'Placed'],
       [65.0, 68.0, 64.0, 'Comm&Mgmt', 'No', 75.0, 'Mkt&Fin', 57.8,
        'Placed'],
       [56.0, 52.0, 52.0, 'Sci&Tech', 'No', 66.0, 'Mkt&HR', 59.43,
        'Not Placed'],
       [85.8, 73.6, 73.3, 'Comm&Mgmt', 'No', 96.8, 'Mkt&Fin', 55.5,
        'Placed']], dtype=object)

### Data Pure?

In [1477]:
def purity_check(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

### Classify

In [1478]:
def node_leaf(data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    leaf = unique_classes[index]

    return leaf

### Potential Splits?

In [1479]:
def get_potential_splits(data):
    potential_splits = {}
    _,n_columns = data.shape
    for column_index in range(n_columns - 1):
        values = data[:, column_index]
        unique_values = np.unique(values)

        potential_splits[column_index] = unique_values
    return potential_splits

### split data

In [1480]:
def split_data(data, split_columns, split_values):
    split_column_values = data[:, split_columns]

    type_of_feature = FEATURE_TYPES[split_columns]
    if type_of_feature == "continous":
        data_below = data[split_column_values <= split_values]
        data_above = data[split_column_values > split_values]
    else:
        data_below = data[split_column_values == split_values]
        data_above = data[split_column_values != split_values]
    
    return data_below, data_above

### Lowest Overall Entropy 

In [1481]:
def calculate_entropy(data):
    label_columns = data[:, -1]
    _, counts = np.unique(label_columns, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

In [1482]:
def calculate_overall_metric(data_below, data_above):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy = (p_data_below * calculate_entropy(data_below)
                       + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [1483]:
def determine_best_split(data, potential_splits):
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_columns=column_index, split_values=value)
            current_overall_entropy = calculate_overall_metric(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
        
    return best_split_column, best_split_value

# Decision Tree Algorithm

### Determine Type of Feature

In [1484]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "status":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
        
    return feature_types

### Algorihtm

In [1485]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
        
        
    # base cases
    if (purity_check(data)) or (len(data) < min_samples) or (counter == max_depth):
        leaf = node_leaf(data)
            
        return leaf

        
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_columns, split_values = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_columns, split_values)
            
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            leaf = node_leaf(data)
            return leaf
            
        # determine question
        feature_name = COLUMN_HEADERS[split_columns]
        type_of_feature = FEATURE_TYPES[split_columns]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_values)
                
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_values)
            
        # instantiate sub-tree
        sub_tree = {question: []}
            
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
            
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
            
        return sub_tree

In [1486]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'work_experience = Yes': [{'ssc_percentage <= 52.0': ['Not Placed',
                                                       {'degree_percentage <= 58.0': ['Not '
                                                                                      'Placed',
                                                                                      'Placed']}]},
                           {'hsc_percentage <= 64.0': ['Not Placed',
                                                       {'hsc_percentage <= 47.0': ['Not '
                                                                                   'Placed',
                                                                                   'Placed']}]}]}


## classification

In [1487]:
example = test_df.iloc[0]
example

ssc_percentage              69.0
hsc_percentage              73.0
degree_percentage           65.0
undergrad_degree       Comm&Mgmt
work_experience               No
emp_test_percentage         70.0
specialisation           Mkt&Fin
mba_percent                57.31
status                    Placed
Name: 98, dtype: object

In [1488]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
        
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
        
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [1489]:
classify_example(example, tree)

'Placed'

# Calculate Accuracy

In [1490]:
def make_predict(df,tree):
    if len(df) != 0:
        prediction = df.apply(classify_example, args=(tree,), axis=1)
    else:
        prediction = pd.Series()

    return prediction

In [1491]:
def calculate_accuracy(df, tree):

    predictions = make_predict(df, tree)
    predictions_correct = predictions == df.status
        
    accuracy = predictions_correct.mean()
        
    return accuracy

In [1492]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.8636363636363636